# GLayout Testing Notebook

This notebook serves as a verification step that everything works as expected.
To correctly evaluate the outputs, you should open klayout (which already has klive installed) so each cell can be displayed on it.

~~~bash
klayout -t
~~~

## Basic Example: Via Instantiation

In [28]:
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk
from glayout.flow.pdk.mappedpdk import MappedPDK
from gdsfactory import Component
from gdsfactory.components import rectangle


def via(PDK: MappedPDK):
    via_dimension = PDK.get_grule("via1")["width"]
    metal1_dimension = (
        via_dimension + 2 * PDK.get_grule("via1", "met1")["min_enclosure"]
    )
    metal2_dimension = (
        via_dimension + 2 * PDK.get_grule("via1", "met2")["min_enclosure"]
    )

    via_layer = PDK.get_glayer("via1")
    metal1_layer = PDK.get_glayer("met1")
    metal2_layer = PDK.get_glayer("met2")

    top_level = Component(name="via")
    top_level << rectangle(
        size=(via_dimension, via_dimension), layer=via_layer, centered=True
    )
    top_level << rectangle(
        size=(metal1_dimension, metal1_dimension), layer=metal1_layer, centered=True
    )
    top_level << rectangle(
        size=(metal2_dimension, metal2_dimension), layer=metal2_layer, centered=True
    )

    return top_level


via_component = via(PDK=sky130_mapped_pdk)
via_component.show()

2024-06-26 17:17:34.880 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/via$8.gds"}


## Some Primitives

In [29]:
from glayout.flow.primitives.fet import nmos
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk


comp = nmos(pdk=sky130_mapped_pdk, fingers=5)
print(comp.info["netlist"].generate_netlist())

comp.show()

/usr/local/lib/python3.10/dist-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 1 in 'Unnamed_41e5dcad'
  gdspath = component.write_gds(
2024-06-26 17:17:43.980 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_41e5dcad.gds"}




.subckt NMOS D G S B l=0.15 w=3 m=5 dm=1 
XMAIN   D G S B sky130_fd_pr__nfet_01v8 l={l} w={w} m={m}
XDUMMY1 B B B B sky130_fd_pr__nfet_01v8 l={l} w={w} m={dm}
XDUMMY2 B B B B sky130_fd_pr__nfet_01v8 l={l} w={w} m={dm}
.ends NMOS


In [30]:
from glayout.flow.primitives.mimcap import mimcap
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk

comp = mimcap(pdk=sky130_mapped_pdk, size=[20.0, 5.0])
print(comp.info["netlist"].generate_netlist())

comp.show()

/usr/local/lib/python3.10/dist-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 1 in 'Unnamed_6d7db294'
  gdspath = component.write_gds(
2024-06-26 17:17:47.728 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_6d7db294.gds"}




.subckt MIMCap V1 V2 l=1 w=1
X1 V1 V2 sky130_fd_pr__cap_mim_m3_1 l={l} w={w}
.ends MIMCap


In [31]:
from glayout.flow.primitives.guardring import tapring
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk

comp = tapring(pdk=sky130_mapped_pdk, enclosed_rectangle=[10.0, 5.0])

comp.show()

/usr/local/lib/python3.10/dist-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 1 in 'Unnamed_bf4af018$1'
  gdspath = component.write_gds(
2024-06-26 17:17:51.053 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_bf4af018$1.gds"}


## Some Designs

### Current Mirror

In [32]:
from glayout.flow.primitives.guardring import tapring
from glayout.flow.primitives.fet import pmos
from glayout.flow.pdk.util.comp_utils import evaluate_bbox, prec_center
from glayout.flow.pdk.mappedpdk import MappedPDK
from glayout.flow.routing.straight_route import straight_route
from glayout.flow.routing.c_route import c_route
from gdsfactory import Component
from glayout.flow.pdk.gf180_mapped import gf180_mapped_pdk
# Move to sky130 please


def currentMirror(pdk: MappedPDK):
    currMirrComp = Component()
    pfet_ref = pmos(pdk, with_substrate_tap=False, with_dummy=(False, True))
    pfet_mir = pmos(pdk, with_substrate_tap=False, with_dummy=(True, False))
    cref_ref = currMirrComp << pfet_ref
    cmir_ref = currMirrComp << pfet_mir
    pdk.util_max_metal_seperation()
    cref_ref.movex(evaluate_bbox(pfet_mir)[0] + pdk.util_max_metal_seperation())
    tap_ring = tapring(
        pdk,
        enclosed_rectangle=evaluate_bbox(
            currMirrComp.flatten(),
            padding=pdk.get_grule("nwell", "active_diff")["min_enclosure"],
        ),
    )
    shift_amount = -prec_center(currMirrComp.flatten())[0]
    tring_ref = currMirrComp << tap_ring
    tring_ref.movex(destination=shift_amount)
    currMirrComp << straight_route(
        pdk,
        cref_ref.ports["multiplier_0_source_E"],
        cmir_ref.ports["multiplier_0_source_E"],
    )
    currMirrComp << straight_route(
        pdk,
        cref_ref.ports["multiplier_0_gate_E"],
        cmir_ref.ports["multiplier_0_gate_E"],
    )
    currMirrComp << c_route(
        pdk,
        cref_ref.ports["multiplier_0_gate_E"],
        cref_ref.ports["multiplier_0_drain_E"],
    )
    return currMirrComp


cm = currentMirror(gf180_mapped_pdk)
cm.show()

2024-06-26 17:17:53.931 | INFO     | gdsfactory.pdk:activate:337 - 'gf180' PDK is now active
/usr/local/lib/python3.10/dist-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 4 in 'Unnamed_05ebaa75'
  gdspath = component.write_gds(
2024-06-26 17:18:00.833 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/Unnamed_05ebaa75.gds"}


### Differential Pair

Here we can see the power of GLayout

1. Differential Pair
2. Active Load

In [36]:
from glayout.flow.blocks.diff_pair import diff_pair
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk

comp = diff_pair(pdk=sky130_mapped_pdk)
print(comp.info["netlist"].generate_netlist())

comp.show()

2024-06-26 17:22:53.298 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/diff_pair_471429cf$2.gds"}


.subckt NMOS D G S B l=0.15 w=3 m=4 dm=1 
XMAIN   D G S B sky130_fd_pr__nfet_01v8 l={l} w={w} m={m}
XDUMMY1 B B B B sky130_fd_pr__nfet_01v8 l={l} w={w} m={dm}
.ends NMOS

.subckt DIFF_PAIR VP VN VDD1 VDD2 VTAIL B
X0 VDD1 VP VTAIL B NMOS l=0.15 w=3 m=2.0 dm=1
X1 VDD2 VN VTAIL B NMOS l=0.15 w=3 m=2.0 dm=1
.ends DIFF_PAIR


In [37]:
from glayout.flow.pdk.sky130_mapped import sky130_mapped_pdk
from glayout.flow.blocks.differential_to_single_ended_converter import (
    differential_to_single_ended_converter,
)

comp = differential_to_single_ended_converter(
    pdk=sky130_mapped_pdk, rmult=1, half_pload=[2, 0.5, 1], via_xlocation=0
)
print(comp.info["netlist"].generate_netlist())

comp.show()

/usr/local/lib/python3.10/dist-packages/gdsfactory/show.py:40: UserWarning: Unnamed cells, 4 in 'shared_gate_components$1'
  gdspath = component.write_gds(
2024-06-26 17:23:10.564 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/shared_gate_components$1.gds"}




.subckt DIFF_TO_SINGLE VIN VOUT VSS VSS2 l=0.5 w=2 mt=8 mb=2 
XTOP1 V1   VIN VSS  VSS sky130_fd_pr__pfet_01v8 l={l} w={w} m={mt}
XTOP2 VSS2 VIN VSS  VSS sky130_fd_pr__pfet_01v8 l={l} w={w} m={mt}
XBOT1 VIN  VIN V1   VSS sky130_fd_pr__pfet_01v8 l={l} w={w} m={mb}
XBOT2 VOUT VIN VSS2 VSS sky130_fd_pr__pfet_01v8 l={l} w={w} m={mb}
.ends DIFF_TO_SINGLE


## Design of an OTA

In [39]:
from glayout.flow.blocks.opamp import opamp

opamp_params = {
    "half_diffpair_params": (6, 1, 4),
    "diffpair_bias": (6, 2, 4),
    "half_common_source_params": (7, 1, 10, 3),
    "half_common_source_bias": (6, 2, 8, 2),
    "output_stage_params": (5, 1, 16),
    "output_stage_bias": (6, 2, 4),
    "half_pload": (6, 1, 6),
    "mim_cap_size": (12, 12),
    "mim_cap_rows": 3,
    "rmult": 2,
}

comp = opamp(sky130_mapped_pdk, **opamp_params)
print(comp.info["netlist"].generate_netlist())

comp.show()

2024-06-26 17:32:06.339 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.2", "type": "open", "file": "/tmp/gdsfactory/opamp_471429cf$1.gds"}


.subckt NMOS D G S B l=1.0 w=6.0 m=4 dm=1 
XMAIN   D G S B sky130_fd_pr__nfet_01v8 l={l} w={w} m={m}
XDUMMY1 B B B B sky130_fd_pr__nfet_01v8 l={l} w={w} m={dm}
.ends NMOS

.subckt DIFF_PAIR VP VN VDD1 VDD2 VTAIL B
X0 VDD1 VP VTAIL B NMOS l=1.0 w=6.0 m=2.0 dm=1
X1 VDD2 VN VTAIL B NMOS l=1.0 w=6.0 m=2.0 dm=1
.ends DIFF_PAIR

.subckt CMIRROR VREF VCOPY VSS VB l=2.0 w=6.0 m=1 
XA VREF VREF VSS VB sky130_fd_pr__nfet_01v8 l={l} w={w} m={m}
XB VCOPY VREF VSS VB sky130_fd_pr__nfet_01v8 l={l} w={w} m={m}
.ends CMIRROR

.subckt NMOS_1 D G S B l=0.5 w=1 m=5 dm=1 
XMAIN   D G S B sky130_fd_pr__nfet_01v8 l={l} w={w} m={m}
.ends NMOS_1

.subckt INPUT_STAGE VP VN VDD1 VDD2 IBIAS VSS B
X0 VP VN VDD1 VDD2 wire0 B DIFF_PAIR
X1 IBIAS wire0 VSS VSS CMIRROR l=2.0 w=6.0 m=4
X2 VSS VSS VP VSS NMOS_1 l=0.5 w=1 m=2.5 dm=1
X3 VSS VSS VN VSS NMOS_1 l=0.5 w=1 m=2.5 dm=1
.ends INPUT_STAGE

.subckt DIFF_TO_SINGLE VIN VOUT VSS VSS2 l=1.0 w=6.0 mt=8 mb=12 
XTOP1 V1   VIN VSS  VSS sky130_fd_pr__pfet_01v8 l={l} w={w} m